In [1]:
%load_ext autoreload
%autoreload 2

## Training Transformer

In [65]:
import os
while 'notebooks' in os.getcwd():
    os.chdir("..")

import numpy as np
import pandas as pd 
from src.utils import train_test_split, get_sample_weights, get_eval_set
from src.preprocessing import preprocess_data
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
from src.preprocessing import TextDataset
import torch
from torch.utils.data import DataLoader, Dataset
from IPython.display import clear_output
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, roc_auc_score
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, LoggingHandler
import logging
from copy import deepcopy
from sklearn.decomposition import PCA
from huggingface_hub import notebook_login
from sklearn.ensemble import RandomForestClassifier
from peft import get_peft_model, LoraConfig, TaskType
from collections import defaultdict
import transformers
from peft import get_peft_model, LoraConfig, TaskType
import re
from bert_score import BERTScorer
import langid
from src.utils import aggregate_samples, evaluate_model, compute_class_weights, remove_hashtag_links

from transformers import BitsAndBytesConfig

tqdm.pandas()

In [ ]:
train_data, test_data = train_test_split()


  0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:06<00:00,  2.66it/s]


In [5]:
# sample_text = "Threre was a goal, half time, kick-off, full time, penalty, red card, yellow card, or own goal"
# # Define batch size
# batch_size = 1024

# # Initialize lists to store scores
# precisions = []
# recalls = []
# f1_scores = []

# data = pd.concat(train_data.values())

# # Create a progress bar
# for i in tqdm(range(0, len(data), batch_size), desc="Scoring Batches"):
#     # Slice the batch
#     batch = data.iloc[i:i + batch_size]['Tweet'].tolist()
    
#     # Compute BERTScore for the batch
#     P, R, F1 = scorer.score(batch, [sample_text] * len(batch), )
    
#     # Append scores
#     precisions.extend(P.tolist())
#     recalls.extend(R.tolist())
#     f1_scores.extend(F1.tolist())

In [6]:
# all_df = pd.concat(train_data.values())
# all_df['bertscore'] = f1_scores

In [7]:
# train = all_df.groupby(["MatchID", "PeriodID"], as_index=False).apply(lambda x: x.sort_values("bertscore").iloc[-50:])

In [32]:
df = pd.concat(train_data)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", cache_dir = '/Data')
df['tokens'] = df['Tweet'].progress_apply(tokenizer.tokenize)

target_words = "goal goooal score yellow card red overtime time halftime half start end kickoff kick penalty "
target_words = set(tokenizer.tokenize(target_words))

def is_valid_text(t):
    for w in t:
        if w in target_words:
            return True
        
    return False

df['is_valid']= df['tokens'].progress_apply(is_valid_text)
# df['lan'] = df['Tweet'].progress_apply(lambda x : langid.classify(x)[0])

/users/eleves-a/2022/pedro.silva/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 1472980/1472980 [00:01<00:00, 982258.79it/s] 


In [33]:
en_df = df#.query("lan == 'en' ")

In [34]:
possible_indices = set(train_data.keys())

In [ ]:
test_indices = list(np.random.choice(list(possible_indices), size=3, replace = False,))
test_indices = [13,1,18]
all_train_indices = list(possible_indices.difference(set(test_indices)))
val_indices = [1,5,12,19]
# val_indices = list(np.random.choice(all_train_indices, 3, replace=False))
# train_indices = list(set(all_train_indices).difference(set(val_indices)))
train_indices = [0,2,7,11,13,18]


train_df = aggregate_samples(train_indices,df = en_df)
# train_df = get_samples(train_indices, df = train)
test_df = aggregate_samples(test_indices)
val_df = aggregate_samples(val_indices, df = en_df)

/tmp/ipykernel_1488553/2907081575.py:21: FutureWarning: The provided callable <function mean at 0x7f07701ddf80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  return (df.query(f"MatchID in {indices}")).groupby(["MatchID", "PeriodID"]).agg({
/tmp/ipykernel_1488553/2907081575.py:14: FutureWarning: The provided callable <function mean at 0x7f07701ddf80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  return pd.concat(all_df).groupby(["MatchID", "PeriodID"]).agg({
/tmp/ipykernel_1488553/2907081575.py:21: FutureWarning: The provided callable <function mean at 0x7f07701ddf80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead

In [41]:
train_df = remove_hashtag_links(train_df)
test_df = remove_hashtag_links(test_df)
val_df = remove_hashtag_links(val_df)

In [259]:
test_indices

[2, 14, 1]

In [267]:
# K Fold CV


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", cache_dir = '/Data')

device = 'cuda'
final_results = []


train_dataset = TextDataset(
    train_df["Tweet"].tolist(), 
    train_df['ID'].tolist(),
    train_df["EventType"].tolist(), 
    tokenizer
)

val_dataset = TextDataset(
    val_df["Tweet"].tolist(), 
    val_df['ID'].tolist(),
    val_df["EventType"].tolist(), 
    tokenizer
)

test_dataset = TextDataset(
    test_df["Tweet"].tolist(), 
    test_df['ID'].tolist(),
    test_df["EventType"].tolist(), 
    tokenizer
)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", cache_dir = '/Data', num_labels = 2, dropout = 0.4)
# model.resize_position_embeddings(2048)

# model = BertWithExtraFeature(bert_model_name="bert-base-uncased")

# for p in model.distlbert.parameters():
#     p.requires_grad = False
model.to(device)

# model = get_peft_model(base_model, lora_config)
# for param in model.bert.parameters():
#     param.requires_grad = False
    
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4,)

# optimizer = torch.optim.AdamW([
#     {'params': model.bert.parameters(), 'weight_decay': 1e-3},  # Regularize BERT weights
#     {'params': model.fc.parameters(), 'weight_decay': 1e-2}     # Stronger regularization on the classifier
# ], lr=1e-5)

for name, param in model.distilbert.named_parameters():
    if "layer.5" in name or "layer.4" in name:  # Unfreeze last two layers
        param.requires_grad = True

    else:
        param.requires_grad = False

best_model = None
second_best_model = None
best_acc = -1
second_best_acc = -1

n_epochs = 10

labels = train_df["EventType"].tolist()
class_weight = torch.Tensor([0.4, 0.6]).to(device)
# class_weights = compute_class_weights(labels).to(device)

# Define weighted loss function
loss_fn = torch.nn.CrossEntropyLoss(class_weight)

for epoch in range(n_epochs):
    all_preds = []
    all_labels = []
    epoch_loss = 0

    print(train_indices, val_indices)
    for i, batch in enumerate(tqdm(train_dataloader)):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        count = batch['count'].to(device).unsqueeze(dim = -1)

        with torch.autocast( device_type = 'cuda'):
            # outputs = model(input_ids=input_ids, attention_mask=attention_mask, extra_feature = count)
            # loss = loss_fn(outputs, labels.squeeze() )

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels = labels)
            loss = loss_fn(outputs.logits, labels)
            # loss = loss_fn(outputs, labels)
        loss.backward()

        
        optimizer.step()
        optimizer.zero_grad()

        preds = torch.argmax(outputs.logits, dim=1)
        # preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        epoch_loss += loss.item()

    acc_train = accuracy_score(all_labels, all_preds, )
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    conf_matrix = confusion_matrix(all_labels, all_preds)


    
    print(f"---------- Epoch {epoch} ------------")
    print(f"Training Loss : {epoch_loss}\n")
    print(f"Training Accuracy : {acc_train}\n")
    print(f"Training Precision : {precision}\n")
    print(conf_matrix)


    preds, labels = evaluate_model(val_df, val_dataloader, model, extra_feature=False)

    acc = accuracy_score(labels, preds)

    # if acc_train > 0.8 and acc > 0.68:
    #     best_model = deepcopy(model)

    if acc > best_acc:
        second_best_acc = best_acc
        second_best_model = deepcopy(best_model)  # Promote previous best to second best
        best_acc = acc
        best_model = deepcopy(model)
    # elif acc > second_best_acc:
    #     second_best_acc = acc
    #     second_best_model = deepcopy(model)


# Combine results for this fold
# validation_results = pd.DataFrame({
#     "MatchID": validation_data["MatchID"].values,
#     "true_values": labels,
#     "predictions": preds,
# })

# final_results.append(validation_results)

Validation Accuracy : 0.6931818181818182

Validation auc : 0.6933850129198966

[[154  71]
 [ 64 151]]


In [235]:
# from torch.utils.data import DataLoader, Dataset

# class TweetDataset(Dataset):
#     def __init__(self, tweets, tokenizer, max_length=512):
#         self.tweets = tweets
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.tweets)

#     def __getitem__(self, idx):
#         return self.tokenizer(
#             self.tweets[idx],
#             return_tensors='pt',
#             truncation=True,
#             padding='max_length',
#             max_length=self.max_length
#         )

# # Define a dataset

# def get_X(texts):
#     tweet_dataset = TweetDataset(
#         tweets=texts,
#         tokenizer=tokenizer,
#         max_length=512
#     )

#     # Create a DataLoader for batching
#     batch_size = 32  # Adjust batch size based on available GPU memory
#     data_loader = DataLoader(tweet_dataset, batch_size=batch_size)

#     X_train_list = []  # To store processed embeddings

#     # Process in batches
#     with torch.no_grad():
#         for batch in tqdm(data_loader):
#             # Move tokenized input to GPU
#             b = {k: v.squeeze().to("cuda") for k, v in batch.items()}  # Squeeze to match model input shape
            
#             # Model forward pass
#             outputs = best_model.distilbert(**b)
            
#             # Collect the embeddings (modify based on your use case, e.g., logits, hidden states, etc.)
#             X_train_list.append(outputs.last_hidden_state[:,-1,:].to("cpu"))

#     # Concatenate all batches if needed
#     X = torch.cat(X_train_list, dim=0)
    
#     return X

In [268]:
preds, labels = evaluate_model(test_df, test_dataloader, best_model, extra_feature=False)

Validation Accuracy : 0.6846153846153846

Validation auc : 0.6778722030981066

[[105  61]
 [ 62 162]]


## Training the model

In [ ]:
# val_indices = list(np.random.choice(list(train_data.keys()), size=6, replace = False))
# train_indices = list(set(train_data.keys()).difference(set(val_indices)))

val_indices = [1,5,12,19]
# val_indices = list(np.random.choice(all_train_indices, 3, replace=False))
# train_indices = list(set(all_train_indices).difference(set(val_indices)))
train_indices = [0,2,7,11,13,18]



# train_df = get_samples(train_indices, df = train)
train_df = aggregate_samples(train_indices, df = en_df)
val_df = aggregate_samples(val_indices, df = en_df)

/tmp/ipykernel_1488553/2907081575.py:21: FutureWarning: The provided callable <function mean at 0x7f07701ddf80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  return (df.query(f"MatchID in {indices}")).groupby(["MatchID", "PeriodID"]).agg({
/tmp/ipykernel_1488553/2907081575.py:21: FutureWarning: The provided callable <function mean at 0x7f07701ddf80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  return (df.query(f"MatchID in {indices}")).groupby(["MatchID", "PeriodID"]).agg({


In [43]:
train_df = remove_hashtag_links(train_df, )
val_df = remove_hashtag_links(val_df)

In [44]:
train_df['EventType'].value_counts(normalize=True)

EventType
1.0    0.567949
0.0    0.432051
Name: proportion, dtype: float64

In [66]:
val_df['EventType'].value_counts(normalize=True)

EventType
1.0    0.558522
0.0    0.441478
Name: proportion, dtype: float64

In [ ]:
# K Fold CV


tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096", cache_dir = '/Data')
# tokenizer.pad_token = tokenizer.eos_token


device = 'cuda'
final_results = []


train_dataset = TextDataset(
    train_df["Tweet"].tolist(), 
    train_df['ID'].tolist(),
    train_df["EventType"].tolist(), 
    tokenizer,
    train_df.index.get_level_values("MatchID").tolist()

)

val_dataset = TextDataset(
    val_df["Tweet"].tolist(), 
    val_df['ID'].tolist(),
    val_df["EventType"].tolist(), 
    tokenizer,
    val_df.index.get_level_values("MatchID").tolist()
)


train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)


model = AutoModelForSequenceClassification.from_pretrained("allenai/longformer-base-4096", cache_dir = '/Data', num_labels = 2,ignore_mismatched_sizes=True)
# for p in base_model.model.parameters():
#     p.requires_grad = False
model.to(device)

# model = get_peft_model(base_model, lora_config)
# for param in model.bert.parameters():
#     param.requires_grad = False

for name, param in model.longformer.named_parameters():
    if "layer.11" in name:  # Unfreeze last two layers
        param.requires_grad = True

    else:
        param.requires_grad = False
    
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

best_model = None
best_acc = -1

n_epochs = 10

labels = train_df["EventType"].tolist()
class_weights = torch.Tensor([0.4,0.6]).to(device)

# Define weighted loss function
loss_fn = torch.nn.CrossEntropyLoss(class_weights)

for epoch in range(n_epochs):
    all_preds = []
    all_labels = []
    epoch_loss = 0

    print(train_indices, val_indices)
    for i, batch in enumerate(tqdm(train_dataloader)):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        with torch.autocast( device_type = 'cuda'):
            # outputs = model(input_ids=input_ids, attention_mask=attention_mask, extra_feature = count)
            # loss = loss_fn(outputs, labels.squeeze() )

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels = labels)
            loss = loss_fn(outputs.logits, labels)
        loss.backward()

        
        optimizer.step()
        optimizer.zero_grad()

        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        epoch_loss += loss.item()

    acc = accuracy_score(all_labels, all_preds, )
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    conf_matrix = confusion_matrix(all_labels, all_preds)


    
    print(f"---------- Epoch {epoch} ------------")
    print(f"Training Loss : {epoch_loss}\n")
    print(f"Training Accuracy : {acc}\n")
    print(f"Training Precision : {precision}\n")
    print(conf_matrix)


    preds, labels = evaluate_model(val_df, val_dataloader, model)

    acc = roc_auc_score(labels, preds)

    if acc > best_acc:
        best_acc = acc
        best_model = deepcopy(model)


# Combine results for this fold
# validation_results = pd.DataFrame({
#     "MatchID": validation_data["MatchID"].values,
#     "true_values": labels,
#     "predictions": preds,
# })

# final_results.append(validation_results)

/users/eleves-a/2022/pedro.silva/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[0, 2, 7, 11, 13, 18] [1, 5, 12, 19]


  0%|          | 0/49 [00:00<?, ?it/s]

In [38]:
total_test_df = get_eval_set().set_index(["MatchID", "PeriodID"])
test_df = preprocess_data(total_test_df)




100%|██████████| 4/4 [00:01<00:00,  3.49it/s]


In [25]:
# sample_text = "Threre was a goal, half time, kick-off, full time, penalty, red card, yellow card, or own goal"
# # Define batch size
# batch_size = 1024

# # Initialize lists to store scores
# precisions = []
# recalls = []
# # f1_scores = []

# data = test_df

# # Create a progress bar
# for i in tqdm(range(0, len(data), batch_size), desc="Scoring Batches"):
#     # Slice the batch
#     batch = data.iloc[i:i + batch_size]['Tweet'].tolist()
    
#     # Compute BERTScore for the batch
#     P, R, F1 = scorer.score(batch, [sample_text] * len(batch), )
    
#     # Append scores
#     precisions.extend(P.tolist())
#     recalls.extend(R.tolist())
    # f1_scores.extend(F1.tolist())

In [26]:
# test_df['bertscore'] = f1_scores 

In [84]:
# test_df = test_df.groupby(["MatchID", "PeriodID"], as_index=False).apply(lambda x: x.sort_values("bertscore").iloc[-50:])

In [ ]:
# test_df['lan'] = test_df['Tweet'].progress_apply(langid.classify)

100%|██████████| 362397/362397 [04:03<00:00, 1490.06it/s]


In [39]:
# test_df['language'] = test_df['lan'].apply(lambda x: x[0])
test_df['language'] = "en"

test_df_en = test_df.query("language == 'en' ")

In [40]:
processed_test_df = test_df_en.groupby(["MatchID", "PeriodID"]).agg({
    "Tweet":    get_first_texts,
    "ID": len
})

processed_test_df = remove_hashtag_links(processed_test_df)

In [41]:
processed_test_df

Tweet   ID
MatchID PeriodID                                                        
6       0         I Finally get to see Germany play\n   \nFascin...  237
        1         "In a few minutes  of  x ...Can't wait"....Waa...  245
        2         I don't see any team in this World Cup that ca...  254
        3         Future World Cup champions are about to play.....  344
        4         Thing is.. Ozil looks like one of my aquarium ...  456
...                                                             ...  ...
16      125       Someone here just described Group D as "litera...  344
        126       Stat comparison between Serbia & Germany s int...  315
        127       OMFG  just lost??? I'm glad I'm at work then! ...  300
        128       ç›´å‰äºˆæƒ³0-0ã¯å¤–ã‚Œã€‚ã‚»ãƒ«ãƒ“ã‚¢å„ªä½ã...  273
        129       :)))) Nije mala, nije mala...   !!!!!!\nFive h...  241

[516 rows x 2 columns]

In [42]:
test_dataset = TextDataset(
    processed_test_df["Tweet"].tolist(), 
    processed_test_df['ID'].tolist(), 
    None,
    tokenizer,
    [0] * 516
)

test_dataloader = DataLoader(test_dataset, batch_size=32)

In [ ]:
preds, labels, probas = evaluate_model(processed_test_df, test_dataloader, best_model, use_labels=False)

 65%|██████▍   | 11/17 [00:28<00:17,  2.85s/it]

In [44]:
processed_test_df['EventType'] = preds

In [45]:
processed_test_df['EventType'].value_counts(normalize = True)

EventType
0    0.614341
1    0.385659
Name: proportion, dtype: float64

In [35]:
pd.merge(
    total_test_df,
    test_df[["Tweet", "EventType"]],
    left_index=True,
    right_index=True

)[['EventType','ID']]\
    .drop_duplicates("ID")\
    .set_index("ID")\
    .to_csv("predictions_2.csv")

KeyError: "['EventType'] not in index"

EventType
0    0.831609
1    0.168391
Name: proportion, dtype: float64